# Principal Component Analysis

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
simpsons = pd.read_csv('simpsons_origineel.csv')
simpsons.info()

In [ ]:
simpsons.index = simpsons.naam # zie je wat dit doet?
simpsons.drop(columns=['naam'], inplace=True)
simpsons

## Eenvoudige projectie

In [ ]:
kleur = simpsons.geslacht
kleur = kleur.str.replace('M', 'blue')
kleur = kleur.str.replace('V', 'pink')

In [ ]:
fig,ax = plt.subplots()
ax.scatter(simpsons.gewicht, simpsons.leeftijd, c=kleur)
ax.set_xlabel('gewicht')
ax.set_ylabel('leeftijd')
#fig.show()

## Data controleren

In [ ]:
kolommen = [ 'haarlengte', 'gewicht', 'leeftijd']
data = simpsons[kolommen]
data

In [ ]:
data.corr()

In [ ]:
data.corr().abs().mean().mean()

## PCA uitvoeren

Eerst z-scores berekenen:

In [ ]:
scaler = StandardScaler()
z_scores = scaler.fit_transform(data)

Nu kan je de PCA uitvoeren:

In [ ]:
pca_algo = PCA(n_components=3)  # de parameter kan ook de minimum verklaarde variantie bepalen
components = pca_algo.fit_transform(z_scores)
nieuwe_data = pd.DataFrame(data=components, columns=['pca1', 'pca2', 'pca3'])
nieuwe_data

Op welke coördinaten wordt Homer dus gemapt?

In [ ]:
pca_algo.transform([[0, 250, 36]]) # dit werkt niet omdat de data eerst genormaliseerd moet worden

In [ ]:
z = scaler.transform([[0, 250, 36]]) # let op: dubbele array!
pca_algo.transform(z)

Welke Simpson zou op 1, 1, 1 gemapt worden?

In [ ]:
original = pca_algo.inverse_transform([[1, 1, 1]])
simpson = scaler.inverse_transform(original)
print(simpson)

Verklaarde variantie per principal component:

In [ ]:
pca_algo.explained_variance_ratio_

Cumulatief:

In [ ]:
pca_algo.explained_variance_ratio_.cumsum()

Bi-plot (selecteer enkel de eerste twee principal components):

In [ ]:
fig, ax = plt.subplots()
ax.scatter(nieuwe_data['pca1'], nieuwe_data['pca2'], c=kleur)
for i in range(len(nieuwe_data)):
    ax.text(nieuwe_data['pca1'][i], nieuwe_data['pca2'][i], s=data.index[i], fontsize=15)
ax.set_xlabel('Principal component 1')
ax.set_ylabel('Principal component 2')
# fig.show()

## Alternatief

In [ ]:
import requests # foutje in pca library: daardoor moet je deze package handmatig installeren
from pca import pca

In [ ]:
model = pca(normalize=True, n_components=3)
pca_result = model.fit_transform(data, verbose=False)
print(pca_result['loadings']) # matrix

Alle nieuwe coördinaten voor de Simpsons:

In [ ]:
print(pca_result['PC'])

In [ ]:
# let op: vanaf versie 2.8.1 voegt dit een rij toe aan de data
print(model.transform([[0, 250, 36]], verbose=False))

In [ ]:
# we moeten de rij dus terug verwijderen (deze heeft index waarde 'mapped')
pca_result['PC'].drop('mapped', axis=0, inplace=True)

In [ ]:
print(pca_result['variance_ratio']) # verklaarde variantie

In [ ]:
print(pca_result['explained_var']) # cumulatieve som van verklaarde varianties

In [ ]:
model.plot()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(pca_result['PC']['PC1'], pca_result['PC']['PC2'], c=kleur)
ax.set_xlabel('Principal component 1')
ax.set_ylabel('Principal component 2')
# fig.show()

In [ ]:
model.biplot()